In [1]:
import numpy as np
import scipy.sparse as sparse
from scipy.linalg import expm, norm
from scipy.io import mmread
from timeit import default_timer as timer

In [2]:
# create adjacency matrix as array

A_array = np.array([[0, 0, 1, 0, 0], [0, 0, 0, 1, 1], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1], [0, 1, 0, 1, 0]])
A_array

array([[0, 0, 1, 0, 0],
       [0, 0, 0, 1, 1],
       [1, 0, 0, 1, 0],
       [0, 1, 1, 0, 1],
       [0, 1, 0, 1, 0]])

In [3]:
# create as sparse matrix

A = sparse.csc_matrix(A_array)
print(A)

  (2, 0)	1
  (3, 1)	1
  (4, 1)	1
  (0, 2)	1
  (3, 2)	1
  (1, 3)	1
  (2, 3)	1
  (4, 3)	1
  (1, 4)	1
  (3, 4)	1


In [4]:
# Arnoldi algorithm

# matrix A for which exp(A)b is of interest, n x n
# b initial vector to be used, length n
# m, the produced Krylov subspace will have dimension m

m = 4

def arnoldi_iteration(A, m: int):
    b = np.ones(A.shape[0]) # b default to be 1_n
    n = A.shape[0]
    h = np.zeros((m + 1, m)) # to become the m x m upper Hessenberg matrix consisting of the coefficients h_ij
    V = np.zeros((n, m + 1)) # to become the orthonormal basis V_m = [v_1, v_2, ..., v_m]
    v = b / norm(b) # makes v a unit 2-norm vector    
    V[:, 0] = v # use v as the first Krylov vector
    
    for j in range(m):
        w = A @ v  # compute candidate vector
        
        for i in range(j + 1):
            h[i, j] = V[:,i] @ w # h_ij-th element is product of v_i and w
            w = w - h[i, j] * V[:, i] # modified Gram-Schmidt
            
        h[j + 1, j] = norm(w)
        
        zero = 1e-12 # small value used as h_ij = 0 threshold
        if h[j + 1, j] > zero: # if nonzero add v to the basis
            v = w / h[j + 1, j]
            V[:, i + 1] = v
        else: 
            return V, h 
        # print('step',j,'out of',m) # to check how far along algorithm is for larger m
    return V, h
            
            

In [5]:
# Arnoldi approximation

# exp(A)v ~ beta x V_m x exp(H_m) x e_1 (m-space 1st unit vector [1, 0, 0,..., 0])

# get results 

Vm1, Hm1 = arnoldi_iteration(A, m)

Vm, Hm = sparse.csc_matrix(Vm1[:,0:m]), sparse.csc_matrix(Hm1[0:m,0:m])

In [6]:
# checks

print("check Arnoldi relation", A @ Vm - Vm @ Hm) # should be zero matrix (okay)

orthog = Vm.transpose() @ Vm
print("check orthogonality", orthog.todense()) # should be identity (okay)

check Arnoldi relation   (2, 3)	-2.220446049250313e-16
  (3, 3)	-1.1102230246251565e-16
  (4, 3)	-2.220446049250313e-16
  (0, 3)	-1.1102230246251565e-16
  (1, 3)	-2.220446049250313e-16
check orthogonality [[ 1.00000000e+00  5.55111512e-17  1.38777878e-16 -1.55431223e-16]
 [ 5.55111512e-17  1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.38777878e-16  0.00000000e+00  1.00000000e+00 -2.03960993e-17]
 [-1.55431223e-16  0.00000000e+00 -2.03960993e-17  1.00000000e+00]]


In [7]:
def a_approximation(A, Vm, Hm, m):
    # get beta = ||v||_2
    b = np.ones(A.shape[0])
    beta = norm(b)

    # get unit vector
    e1 = np.zeros((m,1))
    e1[0] = 1
    e1 = sparse.csc_matrix(e1)
    
    X = beta * Vm @ expm(Hm) @ e1
    return X

In [8]:
# check Vm
np.around(sparse.csc_matrix.toarray(Vm), decimals = 3)

array([[ 0.447, -0.707, -0.365,  0.408],
       [ 0.447,  0.   ,  0.548, -0.   ],
       [ 0.447,  0.   , -0.365, -0.816],
       [ 0.447,  0.707, -0.365,  0.408],
       [ 0.447,  0.   ,  0.548, -0.   ]])

In [9]:
# check Hm
np.around(sparse.csc_matrix.toarray(Hm), decimals = 3)

array([[ 2.   ,  0.632,  0.   , -0.   ],
       [ 0.632, -0.   ,  0.775, -0.   ],
       [ 0.   ,  0.775,  0.333,  0.745],
       [ 0.   ,  0.   ,  0.745, -1.333]])

In [10]:
# arnoldi approximation
X = a_approximation(A,Vm,Hm, m).todense()
print('Arnoldi approximation\n',X)

Arnoldi approximation
 [[ 4.22592057]
 [ 9.07757401]
 [ 7.56993527]
 [11.44518816]
 [ 9.07757401]]


In [11]:
# expAb calulated directly
b = np.ones(A.shape[0])
exact = expm(A) @ sparse.csc_matrix(b.reshape(5,1))
print('Exact',exact.todense())

Exact [[ 4.22592057]
 [ 9.07757401]
 [ 7.56993527]
 [11.44518816]
 [ 9.07757401]]
